In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

# 주피터 노트북에서 기존에 생성되었던 디폴트 그래프를 리셋해줌
tf.reset_default_graph() 

np.random.seed(20171101)
tf.set_random_seed(20171101)

# 데이터를 로드함
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# 첫번째 레이어의 필터 개수 정의
num_filters1 = 32

# 784열의 1차원 벡터를 28*28의 2차원 행렬로 만들어줌
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

# 5*5*1 필터 num_filters1(32)개의 초기값을 난수 생성
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
# 정의한 필터를 가로,세로 1 stride로 인풋 이미지에 적용
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')
# bias 값 정의
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))

# activation function으로 ReLu함수를 사용
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

# 2*2 사이즈로 묶어서 가로,세로 2 strides로 맥스 풀링 => 14*14*1 행렬 32개가 리턴
h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

# 두번째 레이어의 필터 개수 정의
num_filters2 = 64


W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

# 2*2 사이즈로 묶어서 가로,세로 2 strides로 맥스 풀링 => 7*7*1 행렬 64개가 리턴 
h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

# fully connected layer (64개의 7*7 행렬을 1차원으로 변환)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

# 입력값의 개수 (7*7*64)
num_units1 = 7*7*num_filters2
# 뉴런의 개수 
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))

# 1024개의 뉴런으로 학습을 한 후 activation function으로 ReLu 사용
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

# 연결 비율 정의 후 드랍 아웃 실행
keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

# 드랍 아웃시킨 결과를 가지고 소프트맥스 함수를 이용하여 10개의 카테고리(0~9)로 분류
w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(hidden2_drop, w0) + b0
p = tf.nn.softmax(k)

# 코스트 함수 정의
t = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=k,labels=t)) 
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# 트레이닝 시작
i = 0
for i in range(501):
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 100 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) / 4 * c
            end = len(mnist.test.labels) / 4 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

saver.save(sess, 'cnn_session')
sess.close()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Step: 0, Loss: 9.631109, Accuracy: 0.118200
Step: 100, Loss: 1.493048, Accuracy: 0.903100
Step: 200, Loss: 1.044419, Accuracy: 0.923300
Step: 300, Loss: 0.779109, Accuracy: 0.946200
Step: 400, Loss: 0.689028, Accuracy: 0.952500
Step: 500, Loss: 0.582306, Accuracy: 0.958800


In [2]:
!ls cnn_session*

cnn_session.data-00000-of-00001 cnn_session.meta
cnn_session.index
